# Sentinel-1 

We want to use Sentinel-1 radar images to detect barges and ships when there is cloud coverage on the Planet Labs satellite imagery.

In [1]:
from sentinelhub import SHConfig
from sentinelhub import WmsRequest, WcsRequest, MimeType, CRS, BBox, DataCollection

# You must set instance_id, sh_client_id, and sh_client_secret in a local config file (see documentation)

config = SHConfig()
config

SHConfig(
  instance_id='aa86d4ad-9b9d-4154-9a1b-b119e6f68d69',
  sh_client_id='36bcd302-fd3b-4488-941a-54441bb8db18',
  sh_client_secret='el2)!^a,aUrjX-}1U/m.4|M;pw.3+frg93&02Q*Q',
  sh_base_url='https://services.sentinel-hub.com',
  geopedia_wms_url='https://service.geopedia.world',
  geopedia_rest_url='https://www.geopedia.world/rest',
  aws_access_key_id='',
  aws_secret_access_key='',
  aws_metadata_url='https://roda.sentinel-hub.com',
  aws_s3_l1c_bucket='sentinel-s2-l1c',
  aws_s3_l2a_bucket='sentinel-s2-l2a',
  opensearch_url='http://opensearch.sentinel-hub.com/resto/api/collections/Sentinel2',
  max_wfs_records_per_query=100,
  max_opensearch_records_per_query=500,
  max_download_attempts=4,
  download_sleep_time=5,
  download_timeout_seconds=120,
  number_of_download_processes=1
)

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt

from sentinelhub import MimeType, CRS, BBox, SentinelHubRequest, SentinelHubDownloadClient, \
    DataCollection, bbox_to_dimensions, DownloadRequest

def plot_image(image, save_dir=None, factor=1.0, clip_range=None, show=True, **kwargs):
    """
    Utility function for plotting RGB images.
    """
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 15))

    if clip_range is not None:
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
    else:
        ax.imshow(image * factor, **kwargs)
    ax.set_xticks([])
    ax.set_yticks([])

    if save_dir:
        fig.savefig(save_dir)

    if not show:
        plt.close()

dir_path = os.getcwd()

In [4]:
print('Supported DataCollections:\n')
for collection in DataCollection.get_available_collections():
    print(collection)

DataCollection.SENTINEL1

Supported DataCollections:

DataCollection.SENTINEL2_L1C
DataCollection.SENTINEL2_L2A
DataCollection.SENTINEL1
DataCollection.SENTINEL1_IW
DataCollection.SENTINEL1_IW_ASC
DataCollection.SENTINEL1_IW_DES
DataCollection.SENTINEL1_EW
DataCollection.SENTINEL1_EW_ASC
DataCollection.SENTINEL1_EW_DES
DataCollection.SENTINEL1_EW_SH
DataCollection.SENTINEL1_EW_SH_ASC
DataCollection.SENTINEL1_EW_SH_DES
DataCollection.DEM
DataCollection.DEM_MAPZEN
DataCollection.DEM_COPERNICUS_30
DataCollection.DEM_COPERNICUS_90
DataCollection.MODIS
DataCollection.LANDSAT8
DataCollection.SENTINEL5P
DataCollection.SENTINEL3_OLCI
DataCollection.SENTINEL3_SLSTR


<DataCollection.SENTINEL1: DataCollectionDefinition(
  api_id: S1GRD
  catalog_id: sentinel-1-grd
  wfs_id: DSS3
  collection_type: Sentinel-1
  sensor_type: C-SAR
  processing_level: GRD
  orbit_direction: BOTH
  is_timeless: False
  has_cloud_coverage: False
)>

## Barge locations from drone images, 3/25-3/28
This is where the barges are (see https://docs.google.com/spreadsheets/d/1GtfuzJnN7G5481bDDwzcZca5_KApqE64pKJxWUwG9wA/edit?usp=sharing):
* 9.620842°
* 64.94333°


* 9.618517
* 64.942


* 9.516611°
* 64.81439°


* 9.517417°
* 64.81469°

### Get list of bounding box center coordinates that intersect the river

In [5]:
import fiona
import cv2
import rasterio
import sys

from shapely.geometry import mapping, Point, Polygon
from shapely.ops import cascaded_union
from shapely.ops import transform

from rasterio.plot import show, reshape_as_image
from rasterio.transform import Affine
from rasterio.mask import mask
from rasterio.features import rasterize

geojson_file_name = 'aoi-1 copy 2.geojson'
geojson_file_path = os.path.join(dir_path, 'geojson/search-AOIs/' + geojson_file_name)

with fiona.open(geojson_file_path, "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

# Some params (in lat-long degs)
tile_size = 0.02
overlap = 0.003

# Coordinates list
coords = []

for polygon in shapes:
#     print(polygon)
    polygon_coords = polygon['coordinates'][0]
    min_long = min(polygon_coords, key=lambda x: x[0])[0]
    max_long = max(polygon_coords, key=lambda x: x[0])[0]
    min_lat = min(polygon_coords, key=lambda x: x[1])[1]
    max_lat = max(polygon_coords, key=lambda x: x[1])[1]
    print('long min: {}, max: {}; lat min: {}, max: {}'.format(min_long, max_long, min_lat, max_lat))

    river_poly = Polygon([[p[0], p[1]] for p in polygon_coords])

    # Start from min long and min lat and increase iteratively
    curr_long = min_long
    curr_lat = min_lat
    while curr_lat < max_lat + tile_size:  # Allow for running over edge if in between
        curr_long = min_long  # Reset long
        while curr_long < max_long + tile_size:
            # Create square polygon
            square = Polygon([(curr_long, curr_lat), (curr_long + tile_size, curr_lat),
                              (curr_long, curr_lat + tile_size), (curr_long + tile_size, curr_lat + tile_size)])

            # Add center of square to coords list if intersects
            if river_poly.intersects(square):
#                 print('appending', (curr_long + tile_size/2, curr_lat + tile_size/2))
                coords.append((curr_long + tile_size/2, curr_lat + tile_size/2))

            curr_long += (tile_size - overlap)
        curr_lat += (tile_size - overlap)  

print(coords)

long min: -65.10871410369853, max: -64.80538845062256; lat min: -9.608546245948771, max: -9.388689592238487
[(-65.09871410369853, -9.598546245948771), (-65.08171410369853, -9.598546245948771), (-65.06471410369853, -9.598546245948771), (-65.04771410369854, -9.598546245948771), (-65.03071410369854, -9.598546245948771), (-64.96271410369856, -9.598546245948771), (-64.94571410369856, -9.598546245948771), (-64.92871410369857, -9.598546245948771), (-64.91171410369857, -9.598546245948771), (-64.89471410369858, -9.598546245948771), (-65.04771410369854, -9.581546245948772), (-65.03071410369854, -9.581546245948772), (-65.01371410369855, -9.581546245948772), (-64.99671410369855, -9.581546245948772), (-64.97971410369856, -9.581546245948772), (-64.96271410369856, -9.581546245948772), (-64.94571410369856, -9.581546245948772), (-64.92871410369857, -9.581546245948772), (-64.91171410369857, -9.581546245948772), (-64.89471410369858, -9.581546245948772), (-64.87771410369858, -9.581546245948772), (-65.0307

### Get satellite tile images from Sentinel 1, masked with geojson

In [6]:
evalscript = """
    //VERSION=3

    return [VV, 4 * VH, VV / VH / 100.0, dataMask]
"""

time_intervals = [('2021-02-19', '2021-02-24'), ('2021-03-25', '2021-03-27')]

# coords = [[-64.81439, -9.516611]]
# coords = [[-64.81439, -9.516611], [-64.81469, -9.517417]]
# coords = [[-64.8095317134856, -9.51310195609826]]

long_half_width = 0.01
lat_half_width = 0.01

for time_interval in time_intervals:
    for coord in coords:
        # Create bounding box for Sentinel-1
        long_corner = coord[0]
        lat_corner = coord[1]
        bbox = BBox([long_corner-long_half_width, lat_corner-lat_half_width,
                     long_corner+long_half_width, lat_corner+lat_half_width], crs=CRS.WGS84)
#         print('bbox', bbox)
        bbox_length = 500.0
        bbox_size = (bbox_length, bbox_length)

        request = SentinelHubRequest(
            evalscript=evalscript,
            input_data=[
                SentinelHubRequest.input_data(
                    data_collection=DataCollection.SENTINEL1_IW_DES,
                    time_interval=time_interval,
                )
            ],
            responses=[
                SentinelHubRequest.output_response('default', MimeType.TIFF)
            ],
            bbox=bbox,
            size=bbox_size,
            config=config
        )

        image = request.get_data()[0]

        res = (long_half_width*2) / bbox_length
        transform = Affine.translation(coord[0] - long_half_width, coord[1] + lat_half_width) * Affine.scale(res, -res)

        time_interval_path = os.path.join(dir_path, '../data/' + time_interval[0] + '_' + time_interval[1])
#         print('time_interval_path', time_interval_path)
        if not os.path.exists(time_interval_path):
            os.makedirs(time_interval_path)

        # One band only
        image_name = 'longlat_{:.3f}_{:.3f}'.format(long_corner, lat_corner)
        image_path = os.path.join(time_interval_path, image_name + '.tif')
#         print('image_path', image_path)

        image = np.expand_dims(np.moveaxis(image.squeeze(), -1, 0)[0], axis=0)
        with rasterio.open(
            image_path,
            'w',
            driver='GTiff',
            height=image.shape[1],
            width=image.shape[2],
            count=1,
            dtype=image.dtype,
            crs='+proj=latlong',
            transform=transform,
        ) as dst:
            dst.write(image)
            
        # All four bands
        image = request.get_data()[0]
        image_name = 'longlat_{:.3f}_{:.3f}'.format(long_corner, lat_corner)
        image_path_full = os.path.join(time_interval_path, image_name + '_full.tif')
#         print('image_path', image_path)

        image = np.moveaxis(image.squeeze(), -1, 0)
        with rasterio.open(
            image_path_full,
            'w',
            driver='GTiff',
            height=image.shape[1],
            width=image.shape[2],
            count=4,
            dtype=image.dtype,
            crs='+proj=latlong',
            transform=transform,
        ) as dst:
            dst.write(image)

        # Load the raster, mask it by the polygon and crop it
        with rasterio.open(image_path) as src:
#             show(src)
            out_image, out_transform = mask(src, shapes, crop=True, nodata=0)

        out_meta = src.meta.copy()
        # Save the resulting raster  
        out_meta.update({"driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
        "transform": out_transform})

        masked_filename = os.path.join(time_interval_path, image_name + '_masked.tif')
        with rasterio.open(masked_filename, "w", **out_meta) as dest:
            dest.write(out_image)

### Masking, image difference, and finding blob

In [7]:
import math

for coord in coords:
    # 1 is older, 2 is newer
    longlat = 'longlat_{:.3f}_{:.3f}'.format(coord[0], coord[1])

    folder1_name = time_intervals[0][0] + '_' + time_intervals[0][1]
    folder2_name = time_intervals[1][0] + '_' + time_intervals[1][1]
    img1_masked_path = os.path.join(dir_path, '../data/' + folder1_name + '/' + longlat + '_masked.tif')
    img2_masked_path = os.path.join(dir_path, '../data/' + folder2_name + '/' + longlat + '_masked.tif')
    img1_path = os.path.join(dir_path, '../data/' + folder1_name + '/' + longlat + '.tif')
    img2_path = os.path.join(dir_path, '../data/' + folder2_name + '/' + longlat + '.tif')
    
    im1_gray = cv2.imread(img1_masked_path, cv2.IMREAD_GRAYSCALE)
    im2_gray = cv2.imread(img2_masked_path, cv2.IMREAD_GRAYSCALE)

    (thresh1, im1_bw) = cv2.threshold(im1_gray, 200, 255, cv2.THRESH_BINARY)
    (thresh2, im2_bw) = cv2.threshold(im2_gray, 200, 255, cv2.THRESH_BINARY)

    # Subtract older from newer image
    sub = cv2.subtract(im2_bw, im1_bw)
#     plt.figure()
#     plt.imshow(im1_bw)
#     plt.figure()
#     plt.imshow(im2_bw)
#     plt.figure()
#     plt.imshow(sub)

    diff_path = os.path.join(dir_path, '../data/diff')
    if not os.path.exists(diff_path):
        os.makedirs(diff_path)

    folder_path = os.path.join(diff_path, folder1_name + '__' + folder2_name + '_' + longlat)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    sub_path = os.path.join(folder_path, 'diff.png')

    cv2.imwrite(sub_path, sub)

    # Read image
    img = cv2.imread(sub_path)
    img = cv2.bitwise_not(img)

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur
    smoothed = cv2.GaussianBlur(gray, (0,0), sigmaX=8, sigmaY=8, borderType = cv2.BORDER_DEFAULT)

    # Do adaptive threshold on gray image
    thresh = cv2.adaptiveThreshold(smoothed, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 65, 10)

    # Set up the SimpleBlobdetector with default parameters.
    params = cv2.SimpleBlobDetector_Params()

    # Change thresholds
    params.minThreshold = 0
    params.maxThreshold = 256

    # Filter by Area.
    params.filterByArea = True
    params.minArea = 5
    params.maxArea = 10000

    # Filter by Color (black=0)
    params.filterByColor = False
    params.blobColor = 0

    # Filter by Circularity
    params.filterByCircularity = False
    params.minCircularity = 0.5
    params.maxCircularity = 1

    # Filter by Convexity
    params.filterByConvexity = False
    params.minConvexity = 0.5
    params.maxConvexity = 1

    # Filter by InertiaRatio
    params.filterByInertia = False
    params.minInertiaRatio = 0
    params.maxInertiaRatio = 1

    # Distance Between Blobs
    params.minDistBetweenBlobs = 0

    # Do detecting
    detector = cv2.SimpleBlobDetector_create(params)

    # Get keypoints
    keypoints = detector.detect(thresh)
    
    # Show im2 in current dir
#     img2_path_full = os.path.join(dir_path, '../data/' + folder2_name + '/' + longlat + '_full.tif')
    im2 = cv2.imread(img2_path)
    cv2.imwrite(os.path.join(folder_path, 'orig.png'), im2)

    # Get keypoint locations and radius
    for keypoint_num, keypoint in enumerate(keypoints):
        x = int(keypoint.pt[0])
        y = int(keypoint.pt[1])
        s = keypoint.size
        r = int(math.floor(s/2))
#         print (x,y,r)

        # Convert x, y, r to long-lat
        long_corner = coord[0]
        lat_corner = coord[1]

        res = (long_half_width*2) / bbox_length
        transform = Affine.translation(long_corner - long_half_width, lat_corner + lat_half_width) * Affine.scale(res, -res)
        boat_long, boat_lat = rasterio.transform.xy(transform, x, y)
        print('boat long: {}, lat: {}'.format(boat_long, boat_lat))

        # Draw blobs
        blobs = cv2.drawKeypoints(thresh, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

        # Save result
        blob_path = os.path.join(folder_path, 'blob' + str(keypoint_num) + '.png')
        cv2.imwrite(blob_path, blobs)

        # Draw circle on original image
        overlayed_path = os.path.join(folder_path, 'overlayed' + str(keypoint_num) + '.png')
        overlayed_im = cv2.circle(im2, (x, y), r, (0, 255, 0), 2)
        cv2.imwrite(overlayed_path, overlayed_im)
        
        # Get actual coord
        diff = (boat_long - coord[0])**2 + (boat_lat - coord[1])**2
        print('lat-long diff:', diff)

boat long: -64.92185410369856, lat: -9.588366245948771
lat-long diff: 9.357200000003543e-05
boat long: -64.90469410369857, lat: -9.575886245948771
lat-long diff: 8.131599999996737e-05
boat long: -64.90521410369857, lat: -9.584646245948772
lat-long diff: 5.186000000003325e-05
boat long: -64.90677410369857, lat: -9.584926245948772
lat-long diff: 3.5828000000047295e-05
boat long: -64.91313410369857, lat: -9.58120624594877
lat-long diff: 2.131999999989494e-06
boat long: -64.89441410369858, lat: -9.576686245948771
lat-long diff: 2.3709600000004765e-05
boat long: -64.89573410369857, lat: -9.576926245948771
lat-long diff: 2.238480000000254e-05
boat long: -64.80629410369859, lat: -9.507286245948773
lat-long diff: 5.088400000005088e-05
